# Library imports, configurations

In [32]:
import os
import re
import s3fs
import pandas as pd
pd.set_option('display.max_columns', None)
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [6]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

# Functions

In [104]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(str(s))
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(str(s).lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
    return df_c

def remove_punctuation_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
    return df_c

def tolower_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(tolower)
    return df_c

# DATA Download

In [8]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])

/tmp/ipykernel_66561/572923163.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2023_df = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_66561/572923163.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2022_df = pd.read_csv(file_in, sep=";")


In [105]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [115]:
offres_df_clean = ( offres_df
    .pipe(remove_accent_from_df,['Métier','Intitulé du poste'])
    .pipe(remove_punctuation_from_df,['Métier','Intitulé du poste'])
    .pipe(tolower_df,['Métier','Intitulé du poste'])
)

In [119]:
offres_df_clean[offres_df_clean['Métier'].str.contains("data scientist",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data scientist",na=False)].head(5)

,Organisme de rattachement,Référence,Versant,Métier,Statut du poste,Nature de l'emploi,Durée du contrat,Intitulé du poste,Localisation du poste,Lieu d'affectation,Niveau d'études,Niveau d'expérience min. requis,Date de vacance de l'emploi,Date de début de publication par défaut,Date de fin de publication par défaut,Date de première publication,Spécialisation,Télétravail,Management,Employeur,Catégorie,Temps Plein,Durée du poste,Langues,Niveaux,Lieu d'affectation (sans géolocalisation),Avis de vacances au JO,Compétences attendues,Documents à transmettre,Nature de contrat
362,Centre Hospitalier Universitaire de Tours,2022-1090986,Fonction Publique Hospitalière,data scientist,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,datascientist centre de donnees cliniques,NaN,"Tours, 71110 Anzy-le-Duc, France",NaN,NaN,02/01/2023,26/12/2022,25/01/2023,26/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),NaN,NaN,NaN,NaN,Tours,NaN,NaN,NaN,NaN
7616,Muséum National d'Histoire Naturelle,2023-1098299,Fonction Publique de l'Etat,data scientist,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,ingenieur en calcul scientifique h f,Paris (75),"75001 Paris, France",NaN,NaN,01/03/2023,09/01/2023,04/02/2023,09/01/2023,NaN,NaN,NaN,Muséum national d'histoire naturelle,Catégorie A (cadre),NaN,NaN,NaN,NaN,Jardin des Plantes,NaN,NaN,NaN,NaN
8887,Direction générale des finances publiques (DGFiP),2023-1099252,Fonction Publique de l'Etat,data scientist,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,ssi dpn dtnum bureau si mission service a l ut...,Seine Saint-Denis (93),NaN,Niveau 6 Licence/diplômes équivalents,Expert,01/01/2023,10/01/2023,09/02/2023,10/01/2023,NaN,Oui,Non,SSI DPN DTNUM BUREAU SI2,Catégorie A (cadre),NaN,NaN,NaN,NaN,Immeuble Le Montaigne 4 Avenue Montaigne 93160...,NaN,NaN,NaN,NaN
8902,Institut national de l'information géographiqu...,2023-1099592,Fonction Publique de l'Etat,data scientist,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,ingenieur conception consolidation et prototyp...,Seine et Marne (77),NaN,Niveau 7 Master/diplômes équivalents,Confirmé,15/02/2023,10/01/2023,09/02/2023,10/01/2023,NaN,Oui,Non,Institut national de l'information géographiqu...,Catégorie A (cadre),Oui,NaN,NaN,NaN,Marne la Vallée,NaN,NaN,NaN,CDD de 3 ans
11965,Ministère de l'éducation nationale et de la je...,2023-1102835,Fonction Publique de l'Etat,data scientist,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,deep b data scientist responsable d analyses d...,Paris (75),NaN,NaN,Débutant,03/04/2023,13/01/2023,13/04/2023,13/01/2023,NaN,Oui,Non,"Direction de l'évaluation, de la prospective ...",Catégorie A (cadre),Oui,NaN,NaN,NaN,DEPP B2.2 61/65 rue Dutot 75015 PARIS,NaN,NaN,NaN,NaN


In [120]:
offres_df_clean[offres_df_clean['Métier'].str.contains("data scientist",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data scientist",na=False)]['Organisme de rattachement'].unique()

array(['Centre Hospitalier Universitaire de Tours',
       "Muséum National d'Histoire Naturelle",
       'Direction générale des finances publiques (DGFiP)',
       "Institut national de l'information géographique et forestière",
       "Ministère de l'éducation nationale et de la jeunesse  -  Ministère de l'enseignement supérieur et de la recherche",
       'Secrétariat Général', 'Administration Centrale MENJ - MESR',
       'Autorité des Marchés Financiers',
       "Agence Régionale de Santé Provence-Alpes-Côte-d'Azur",
       'Agence de Services et de Paiement (ASP)',
       'Structures de coopération territoriale',
       'Direction Interministérielle du Numérique (DINUM)',
       'Agence Nationale de Sécurité du Médicament et des Produits de Santé',
       'Institut de Recherche pour le Développement',
       "Ministère de l'Intérieur",
       'Direction générale des entreprises (DGE)', 'DRSD',
       'Direction des Services Judiciaires',
       "Ministère des Armées - CMG d'Arcu